## Multi-dimensional 0-1 knapsack problem
The solution is encoded as array of length NUM_ITEMS, where each cell is an item and the value inside represents the specific knapsack it's been assigned (from 0 to NUM_KNAPSACK-1) to or -1 if unassigned.

In [ ]:
import numpy as np
from tqdm.notebook import tqdm
from icecream import ic
import matplotlib.pyplot as plt
import math

In [ ]:
NUM_KNAPSACKS = 2
NUM_ITEMS = 10
NUM_DIMENSIONS = 2

In [ ]:
MAX_STEPS = 10000

In [ ]:
VALUES = np.random.randint(0, 100, size=NUM_ITEMS)
WEIGHTS = np.random.randint(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = np.random.randint(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS))

In [ ]:
""" Check if a solution satisfies all constraints """
def valid_solution(solution):
    for kp in range(NUM_KNAPSACKS):
        if np.any(WEIGHTS[solution == kp].sum(axis = 0) > CONSTRAINTS[kp]):
            return False
    return True

In [ ]:
""" Fitness function: sum of values of items in knapsacks if valid """
def fitness(solution):
    return np.sum(VALUES[solution >= 0])

In [ ]:
""" Tweak function: move one item to a different knapsack or unassign it"""
def tweak(solution):
    new_solution = solution.copy()
    # Choose a random item
    i = np.random.randint(0, NUM_ITEMS)
    solution_representations = np.arange(-1, NUM_KNAPSACKS)
    # Choose a new knapsack for the item OR unassign it
    new_value = np.random.choice(solution_representations)
    while new_value == solution[i]:
        new_value = np.random.choice(solution_representations)
    new_solution[i] = new_value
    # If we unassigned an item, assign the backpack to a random unassigned item (if any)
    if new_value == -1:
        old_value = solution[i]
        unassigned_items = np.arange(NUM_ITEMS)[solution == -1]
        if len(unassigned_items) >= 1:
            j = np.random.choice(unassigned_items)
            new_solution[j] = old_value
    return new_solution

In [ ]:
""" Class to store experiment results and plot fitness over time """
class ExperimentResult:
    def __init__(self, name):
        self.name = name
        self.fitness_history = []
        self.best_solution = None
        self.best_fitness = -1
        self.restarts = []
    
    def add_fitness(self, fitness, solution):
        self.fitness_history.append(fitness)
        if fitness > self.best_fitness:
            self.best_fitness = fitness
            self.best_solution = solution.copy()
    
    def plot_fitness(self):
        fig, ax = plt.subplots()
        ax.plot(self.fitness_history)
        if len(self.restarts) > 0:
            for r in self.restarts:
                ax.axvline(r, color='red', linestyle='--', alpha=0.5)
        ax.set_xlabel("Iteration")
        ax.set_ylabel("Fitness")
        ax.set_title(self.name)
        ax.set_xlim(0, len(self.fitness_history))
        ax.set_ylim(0, np.sum(VALUES) * 1.1)
        ax.axhline(np.sum(VALUES), color='green', linestyle='--', alpha=0.5, label='Total maximum value')
        ax.legend(loc="lower right")
        plt.show()
        return ax
    
    def __str__(self):
        header = f"Experiment: {self.name}\n"
        best = f"Best Fitness: {self.best_fitness}\n"
        for k in range(NUM_KNAPSACKS):
            best += f" Knapsack {k}: items {np.where(self.best_solution == k)[0].tolist()}\n"
        best += f" Unassigned items: {np.where(self.best_solution == -1)[0].tolist()}\n"
        return header + best

In [ ]:
""" Simulated Annealing algorithm """
def simulatedAnnealing():
    # start from a valid solution where no items are assigned
    current_solution = np.full(NUM_ITEMS, -1)
    current_obj = fitness(current_solution)
    best_solution = current_solution 
    best_obj = current_obj 
    results = ExperimentResult("Simulated Annealing")
    results.add_fitness(current_obj, current_solution)
    temp = 100
    for step in tqdm(range(MAX_STEPS)):
        new_solution = tweak(current_solution)
        new_obj = fitness(new_solution)
        if valid_solution(new_solution):
            if new_obj >= current_obj:
                current_solution = new_solution
                current_obj = new_obj            
                if new_obj > best_obj:
                    best_solution = new_solution
                    best_obj = new_obj
                
            else:
                diff = new_obj - current_obj
                p = math.exp(diff / temp)

                if np.random.rand() < p:
                    current_solution = new_solution
                    current_obj = new_obj
        temp *= 0.99
        results.add_fitness(current_obj, current_solution)
    return results

In [ ]:
""" Steepest Ascent Hill Climbing algorithm """
def steepestAscentHillClimbing(num_neighbors=5):
    current_solution = np.full(NUM_ITEMS, -1)
    current_obj = fitness(current_solution)
    best_solution = current_solution 
    best_obj = current_obj 
    results = ExperimentResult("Steepest Ascent Hill Climbing")
    results.add_fitness(current_obj, current_solution)
    for step in tqdm(range(MAX_STEPS)):
        neighbors = []
        neighbor_objs = []
        for _ in range(num_neighbors):
            new_solution = tweak(current_solution)
            if valid_solution(new_solution):
                neighbors.append(new_solution)
                neighbor_objs.append(fitness(new_solution))
        if len(neighbors) == 0:
            results.add_fitness(current_obj, current_solution)
            continue
        max_index = np.argmax(neighbor_objs)
        max_obj = neighbor_objs[max_index]
        if max_obj > current_obj:
            current_solution = neighbors[max_index]
            current_obj = max_obj
            if current_obj > best_obj:
                best_solution = current_solution
                best_obj = current_obj
        results.add_fitness(current_obj, current_solution)
    return results

## TEST PROBLEMS

In [ ]:
# Problem 1:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS))

In [ ]:
results = simulatedAnnealing()
results.plot_fitness()
print(results)

In [ ]:
results = steepestAscentHillClimbing()
results.plot_fitness()
print(results)

In [ ]:
# Problem 2:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS,size=(NUM_KNAPSACKS, NUM_DIMENSIONS))

In [ ]:
results = simulatedAnnealing()
results.plot_fitness()
print(results)

In [ ]:
results = steepestAscentHillClimbing()
results.plot_fitness()
print(results)

In [ ]:
# Problem 3:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS))

In [ ]:
MAX_STEPS = 20000

In [ ]:
results = simulatedAnnealing()
results.plot_fitness()
print(results)

In [ ]:
results = steepestAscentHillClimbing(num_neighbors=25)
results.plot_fitness()
print(results)